# CLI Configuration

> Configuration system for multi-library CLI support

In [ ]:
#| default_exp cli.cli_config

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from dataclasses import dataclass, field
from typing import List, Optional, Tuple, Dict, Any
import os

## Configuration Class

Define the configuration structure for library-specific CLI tools:

In [ ]:
#| export
@dataclass
class LibraryConfig:
    """Configuration for a specific library's CLI tool."""
    # Package names
    package_name: str  # e.g., 'cjm_fasthtml_tailwind' or 'cjm_fasthtml_daisyui'
    cli_command: str  # e.g., 'cjm-tailwind-explore' or 'cjm-daisyui-explore'
    display_name: str  # e.g., 'Tailwind' or 'DaisyUI'
    
    # Module discovery configuration
    module_discovery_paths: List[str]  # e.g., ['utilities'] or ['components', 'builders']
    
    # Core package locations
    core_package: str  # e.g., 'cjm_fasthtml_tailwind.core'
    
    # Core utilities to expose
    core_utilities: List[Tuple[str, str]]  # [(name, module_path), ...]
    
    # Import generation
    base_imports: List[str] = field(default_factory=list)  # Additional base imports
    
    # Pattern matching specifics (optional)
    css_class_prefix: Optional[str] = None  # e.g., 'daisy-' for DaisyUI classes
    
    # Test patterns (for finding test examples)
    test_pattern_prefix: str = 'test_'  # Prefix for test functions
    test_pattern_suffix: str = '_examples'  # Suffix for test functions
    
    # Helper patterns
    helper_test_suffix: str = 'helper_examples'  # Suffix for helper test functions
    
    def get_utilities_package(self, module_name: str) -> str:
        """Get the full package path for a utility module."""
        # Find which discovery path contains this module
        for path in self.module_discovery_paths:
            return f"{self.package_name}.{path}.{module_name}"
        # Default to first path
        return f"{self.package_name}.{self.module_discovery_paths[0]}.{module_name}"
    
    def get_test_function_pattern(self, module_name: str, feature: str) -> str:
        """Get the expected test function name for a module and feature."""
        return f"{self.test_pattern_prefix}{module_name}_{feature}_{self.test_pattern_suffix}"
    
    def get_helper_test_function(self, module_name: str) -> str:
        """Get the expected helper test function name for a module."""
        return f"{self.test_pattern_prefix}{module_name}_{self.helper_test_suffix}"

## Predefined Configurations

Define configurations for specific libraries:

In [ ]:
#| export
def get_tailwind_config() -> LibraryConfig:
    """Get configuration for cjm-fasthtml-tailwind library."""
    return LibraryConfig(
        package_name='cjm_fasthtml_tailwind',
        cli_command='cjm-tailwind-explore',
        display_name='Tailwind',
        module_discovery_paths=['utilities'],
        core_package='cjm_fasthtml_tailwind.core',
        core_utilities=[
            ('combine_classes', 'cjm_fasthtml_tailwind.core.base'),
            ('get_tailwind_headers', 'cjm_fasthtml_tailwind.core.resources'),
        ],
        base_imports=['from fasthtml.common import *']
    )

In [ ]:
#| export
def get_daisyui_config() -> LibraryConfig:
    """Get configuration for cjm-fasthtml-daisyui library."""
    return LibraryConfig(
        package_name='cjm_fasthtml_daisyui',
        cli_command='cjm-daisyui-explore',
        display_name='DaisyUI',
        module_discovery_paths=['components', 'builders'],
        core_package='cjm_fasthtml_daisyui.core',
        core_utilities=[
            ('combine_classes', 'cjm_fasthtml_daisyui.core.utility_classes'),
            ('get_daisyui_headers', 'cjm_fasthtml_daisyui.core.resources'),
            ('DaisyThemes', 'cjm_fasthtml_daisyui.core.themes'),
        ],
        base_imports=['from fasthtml.common import *'],
        css_class_prefix='daisy-'
    )

## Configuration Management

Functions to manage the active configuration:

In [ ]:
#| export
# Global variable to store the active configuration
_active_config: Optional[LibraryConfig] = None

def set_active_config(config: LibraryConfig) -> None:
    """Set the active library configuration."""
    global _active_config
    _active_config = config

def get_active_config() -> LibraryConfig:
    """Get the active library configuration.
    
    If no configuration is set, attempts to auto-detect based on:
    1. Environment variable FASTHTML_LIB_CONFIG
    2. Package availability
    3. Defaults to Tailwind config
    """
    global _active_config
    
    if _active_config is not None:
        return _active_config
    
    # Try environment variable
    env_config = os.environ.get('FASTHTML_LIB_CONFIG', '').lower()
    if env_config == 'daisyui':
        _active_config = get_daisyui_config()
    elif env_config == 'tailwind':
        _active_config = get_tailwind_config()
    else:
        # Try to detect based on package availability
        try:
            import cjm_fasthtml_daisyui
            _active_config = get_daisyui_config()
        except ImportError:
            try:
                import cjm_fasthtml_tailwind
                _active_config = get_tailwind_config()
            except ImportError:
                # Default to tailwind
                _active_config = get_tailwind_config()
    
    return _active_config

In [ ]:
#| export
def reset_config() -> None:
    """Reset the active configuration to force re-detection."""
    global _active_config
    _active_config = None

## Configuration Factory

Factory function to get configuration by name:

In [ ]:
#| export
def get_config_by_name(name: str) -> Optional[LibraryConfig]:
    """Get a library configuration by name.
    
    Args:
        name: Library name ('tailwind', 'daisyui')
    
    Returns:
        LibraryConfig or None if not found
    """
    configs = {
        'tailwind': get_tailwind_config,
        'daisyui': get_daisyui_config,
    }
    
    factory = configs.get(name.lower())
    return factory() if factory else None

## Configuration Utilities

Helper functions for working with configurations:

In [ ]:
#| export
def list_available_configs() -> List[str]:
    """List all available library configurations."""
    return ['tailwind', 'daisyui']

In [ ]:
#| export
def get_config_info(config: LibraryConfig) -> Dict[str, Any]:
    """Get information about a configuration.
    
    Returns:
        Dictionary with configuration details
    """
    return {
        'library': config.display_name,
        'package': config.package_name,
        'cli_command': config.cli_command,
        'module_paths': config.module_discovery_paths,
        'core_utilities': [name for name, _ in config.core_utilities],
        'has_css_prefix': config.css_class_prefix is not None
    }

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()